In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
train = pd.read_csv('https://raw.githubusercontent.com/AKapich/NLP_Disaster_Tweets/bert/train.csv?token=GHSAT0AAAAAAB76QMSEXTJNMV75QKBMCQ5MZAPBSFA')
test = pd.read_csv('https://raw.githubusercontent.com/AKapich/NLP_Disaster_Tweets/bert/test.csv?token=GHSAT0AAAAAAB76QMSECF6DALBNQPMV2DOUZAPBSOQ')

HTTPError: ignored

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


# Preprocessing

## Tokenization

In [19]:
from nltk.tokenize import word_tokenize

In [20]:
def tokenize(df):
    df['tokenized_text'] =  df['text'].apply(word_tokenize)

In [21]:
tokenize(train)
tokenize(test)

In [22]:
train['tokenized_text']

0       [Our, Deeds, are, the, Reason, of, this, #, ea...
1        [Forest, fire, near, La, Ronge, Sask, ., Canada]
2       [All, residents, asked, to, 'shelter, in, plac...
3       [13,000, people, receive, #, wildfires, evacua...
4       [Just, got, sent, this, photo, from, Ruby, #, ...
                              ...                        
7608    [Two, giant, cranes, holding, a, bridge, colla...
7609    [@, aria_ahrary, @, TheTawniest, The, out, of,...
7610    [M1.94, [, 01:04, UTC, ], ?, 5km, S, of, Volca...
7611    [Police, investigating, after, an, e-bike, col...
7612    [The, Latest, :, More, Homes, Razed, by, North...
Name: tokenized_text, Length: 7613, dtype: object

## Removing punctuation 

In [23]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
def remove_punctuation(df):
    df['only_text'] = df['tokenized_text'].apply(
        lambda row: [word for word in row if word not in punctuation]
    )

In [25]:
remove_punctuation(train)
remove_punctuation(test)

In [26]:
train['only_text'] 

0       [Our, Deeds, are, the, Reason, of, this, earth...
1           [Forest, fire, near, La, Ronge, Sask, Canada]
2       [All, residents, asked, to, 'shelter, in, plac...
3       [13,000, people, receive, wildfires, evacuatio...
4       [Just, got, sent, this, photo, from, Ruby, Ala...
                              ...                        
7608    [Two, giant, cranes, holding, a, bridge, colla...
7609    [aria_ahrary, TheTawniest, The, out, of, contr...
7610    [M1.94, 01:04, UTC, 5km, S, of, Volcano, Hawai...
7611    [Police, investigating, after, an, e-bike, col...
7612    [The, Latest, More, Homes, Razed, by, Northern...
Name: only_text, Length: 7613, dtype: object

## Removing stopwords

In [27]:
from nltk.corpus import stopwords 
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
def remove_stopwords(df):
    df['cleaned_text'] = df['only_text'].apply(
        lambda row: [word for word in row if word not in stop_words]
    ) 

In [29]:
remove_stopwords(train)
remove_stopwords(test)

In [30]:
train['cleaned_text']

0       [Our, Deeds, Reason, earthquake, May, ALLAH, F...
1           [Forest, fire, near, La, Ronge, Sask, Canada]
2       [All, residents, asked, 'shelter, place, notif...
3       [13,000, people, receive, wildfires, evacuatio...
4       [Just, got, sent, photo, Ruby, Alaska, smoke, ...
                              ...                        
7608    [Two, giant, cranes, holding, bridge, collapse...
7609    [aria_ahrary, TheTawniest, The, control, wild,...
7610    [M1.94, 01:04, UTC, 5km, S, Volcano, Hawaii, h...
7611    [Police, investigating, e-bike, collided, car,...
7612    [The, Latest, More, Homes, Razed, Northern, Ca...
Name: cleaned_text, Length: 7613, dtype: object

# Extracting more information from the tweets

In [31]:
train

,id,keyword,location,text,target,tokenized_text,only_text,cleaned_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[Our, Deeds, are, the, Reason, of, this, #, ea...","[Our, Deeds, are, the, Reason, of, this, earth...","[Our, Deeds, Reason, earthquake, May, ALLAH, F..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, ., Canada]","[Forest, fire, near, La, Ronge, Sask, Canada]","[Forest, fire, near, La, Ronge, Sask, Canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[All, residents, asked, to, 'shelter, in, plac...","[All, residents, asked, to, 'shelter, in, plac...","[All, residents, asked, 'shelter, place, notif..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfires, evacua...","[13,000, people, receive, wildfires, evacuatio...","[13,000, people, receive, wildfires, evacuatio..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[Just, got, sent, this, photo, from, Ruby, #, ...","[Just, got, sent, this, photo, from, Ruby, Ala...","[Just, got, sent, photo, Ruby, Alaska, smoke, ..."
...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,"[Two, giant, cranes, holding, a, bridge, colla...","[Two, giant, cranes, holding, a, bridge, colla...","[Two, giant, cranes, holding, bridge, collapse..."
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,"[@, aria_ahrary, @, TheTawniest, The, out, of,...","[aria_ahrary, TheTawniest, The, out, of, contr...","[aria_ahrary, TheTawniest, The, control, wild,..."
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,"[M1.94, [, 01:04, UTC, ], ?, 5km, S, of, Volca...","[M1.94, 01:04, UTC, 5km, S, of, Volcano, Hawai...","[M1.94, 01:04, UTC, 5km, S, Volcano, Hawaii, h..."
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,"[Police, investigating, after, an, e-bike, col...","[Police, investigating, after, an, e-bike, col...","[Police, investigating, e-bike, collided, car,..."


In [32]:
def count_hashtags(df):
    df['#_amount'] = df['tokenized_text'].apply(lambda row: len([word for word in row if word=='#']))

In [33]:
count_hashtags(train)
count_hashtags(test)

In [34]:
def count_mentions(df):
    df['@_amount'] = df['tokenized_text'].apply(lambda row: len([word for word in row if word=='@']))

In [35]:
count_mentions(train)
count_mentions(test)

In [36]:
def count_punctuation(df):
    df['punctuation_amount'] = df['tokenized_text'].apply(
        lambda row: len([word for word in row if word in punctuation])
    )

In [37]:
count_punctuation(train)
count_punctuation(test)

In [38]:
def count_stopwords(df):
    df['stopword_amount'] = df['tokenized_text'].apply(
        lambda row: len([word for word in row if word in stop_words])
    )

In [39]:
count_stopwords(train)
count_stopwords(test)

In [40]:
def count_urls(df):
    df['url_amount'] = df['tokenized_text'].apply(
        lambda row: len([word for word in row if 'http' in word or 'https' in word])
    )

In [41]:
count_urls(train)
count_urls(test)

In [42]:
def mean_word_length(df):
    df['mean_word_length'] = df['only_text'].apply(
        lambda row: round(np.mean([len(word) for word in row]), 3)
    )

In [43]:
mean_word_length(train)
mean_word_length(test)

In [44]:
def word_amount(df):
    df['word_amount'] = df['only_text'].apply(
        lambda row: len(row)
    )

In [45]:
word_amount(train)
word_amount(test)

In [46]:
def unique_word_amount(df):
    df['unique_word_amount'] = df['only_text'].apply(
        lambda row: len(list(set(row)))
    )

In [47]:
unique_word_amount(train)
unique_word_amount(test)

In [48]:
def has_location(df):
    df['has_location']  = df['location'].str.len()>0

In [49]:
has_location(train)
has_location(test)

In [50]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [51]:
def calc_compound_sentiment(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)['compound']

def sentiment(df):
    df['sentiment'] = df['text'].apply(calc_compound_sentiment)

In [56]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

sentiment(train)
sentiment(test)

In [58]:
def uppercase_percentage(text):
    return(sum(1 for c in text if c.isupper())/len(text))

def count_uppercase_percentage(df):
    df['uppercase_percentage'] = df['text'].apply(uppercase_percentage)

In [59]:
count_uppercase_percentage(train)
count_uppercase_percentage(test)

In [60]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train.sample(frac=0.1), title="Twitter Profiling Report", explorative=True)
# profile.to_file("twitter.html")

# Random Forest

In [61]:
y_train = train['target']
X_train = train.drop(['tokenized_text', 'only_text', 'cleaned_text',
                      'keyword', 'location', 'text', 'target', 'id'], axis=1)
X_test = test.drop(['tokenized_text', 'only_text', 'cleaned_text',
                    'keyword', 'location', 'text', 'id'], axis=1)

In [62]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=120)
rf_model.fit(X_train, y_train)
RandomForestClassifier(n_estimators=120)
y_pred = rf_model.predict(X_test)

In [63]:
indices = list(test['id'])
y_pred = pd.DataFrame ({'id':indices,
                        'target':y_pred})

In [64]:
y_pred.to_csv('prediction.csv', index=False)

# Lazypredict

In [17]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.25, random_state=123)

In [ ]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

In [ ]:
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Further analysis

In [66]:
top_target_keywords = train[train.target==1].keyword.value_counts()
top_target_keywords.head(10)

derailment           39
wreckage             39
outbreak             39
debris               37
oil%20spill          37
typhoon              37
evacuated            32
suicide%20bombing    32
rescuers             32
suicide%20bomb       32
Name: keyword, dtype: int64

In [67]:
top_non_target_keywords = train[train.target==0].keyword.value_counts()
top_non_target_keywords.head(10)

body%20bags    40
harm           37
armageddon     37
wrecked        36
ruin           36
deluge         36
explode        35
twister        35
fear           35
siren          35
Name: keyword, dtype: int64

# BERT

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [68]:
train.head()

,id,keyword,location,text,target,tokenized_text,only_text,cleaned_text,#_amount,@_amount,punctuation_amount,stopword_amount,url_amount,mean_word_length,word_amount,unique_word_amount,has_location,uppercase_percentage
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[Our, Deeds, are, the, Reason, of, this, #, ea...","[Our, Deeds, are, the, Reason, of, this, earth...","[Our, Deeds, Reason, earthquake, May, ALLAH, F...",1,0,1,5,0,4.31,13,13,False,0.14
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, ., Canada]","[Forest, fire, near, La, Ronge, Sask, Canada]","[Forest, fire, near, La, Ronge, Sask, Canada]",0,0,1,0,0,4.43,7,7,False,0.13
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[All, residents, asked, to, 'shelter, in, plac...","[All, residents, asked, to, 'shelter, in, plac...","[All, residents, asked, 'shelter, place, notif...",0,0,2,9,0,5.00,22,19,False,0.02
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfires, evacua...","[13,000, people, receive, wildfires, evacuatio...","[13,000, people, receive, wildfires, evacuatio...",1,0,1,1,0,7.00,8,8,False,0.02
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[Just, got, sent, this, photo, from, Ruby, #, ...","[Just, got, sent, this, photo, from, Ruby, Ala...","[Just, got, sent, photo, Ruby, Alaska, smoke, ...",2,0,2,6,0,4.38,16,15,False,0.03


In [8]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

In [9]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [10]:
get_sentence_embeding([
    "that is a disaster",
    "it's all good"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8474296 , -0.24617359,  0.2015963 , ...,  0.24677038,
        -0.5970604 ,  0.89328897],
       [-0.873268  , -0.3235037 , -0.4701092 , ..., -0.33154625,
        -0.6519238 ,  0.9150243 ]], dtype=float32)>

In [11]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [12]:
l1 = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l2 = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l1)

In [13]:
model = tf.keras.Model(inputs=[text_input], outputs = [l2])

In [70]:
train_

,id,keyword,location,text,target,tokenized_text,only_text,cleaned_text,#_amount,@_amount,punctuation_amount,stopword_amount,url_amount,mean_word_length,word_amount,unique_word_amount,has_location,uppercase_percentage
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[Our, Deeds, are, the, Reason, of, this, #, ea...","[Our, Deeds, are, the, Reason, of, this, earth...","[Our, Deeds, Reason, earthquake, May, ALLAH, F...",1,0,1,5,0,4.31,13,13,False,0.14
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, ., Canada]","[Forest, fire, near, La, Ronge, Sask, Canada]","[Forest, fire, near, La, Ronge, Sask, Canada]",0,0,1,0,0,4.43,7,7,False,0.13
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[All, residents, asked, to, 'shelter, in, plac...","[All, residents, asked, to, 'shelter, in, plac...","[All, residents, asked, 'shelter, place, notif...",0,0,2,9,0,5.00,22,19,False,0.02
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfires, evacua...","[13,000, people, receive, wildfires, evacuatio...","[13,000, people, receive, wildfires, evacuatio...",1,0,1,1,0,7.00,8,8,False,0.02
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[Just, got, sent, this, photo, from, Ruby, #, ...","[Just, got, sent, this, photo, from, Ruby, Ala...","[Just, got, sent, photo, Ruby, Alaska, smoke, ...",2,0,2,6,0,4.38,16,15,False,0.03


In [76]:
y_train = train['target']
X_train = train['text']
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.25, random_state=123)

In [78]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [80]:
model.fit(X_train, y_train)

179/179 [==============================] - 2265s 13s/step - loss: 0.6311 - accuracy: 0.6558 - precision: 0.6278 - recall: 0.4976


In [81]:
y_predicted = model.predict(X_test)

 1/60 [..............................] - ETA: 13:26

KeyboardInterrupt: ignored